##  準備

In [1]:
from pathlib import Path
from bs4 import BeautifulSoup
import json
import os
import re
import time
import shutil
import logging
import pandas as pd
!pip install selenium
!pip install beautiful requests
!pip install BeautifulSoup4
from selenium import webdriver
from selenium.webdriver.support.select import Select
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from time import sleep
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.webdriver.common.alert import Alert

pd.set_option('display.max_rows', 100)

  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.
requests 2.22.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.16 which is incompatible.
cookiecutter 1.7.2 requires requests>=2.23.0, but you have requests 2.22.0 which is incompatible.
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.16
    Uninstalling urllib3-1.26.16:
      Successfully uninstalled urllib3-1.26.16
ERROR: pip's dependency resolv

In [2]:
hard_coded_colleage_list =[
    {"univ_name":"大阪工業大学","url":"https://www.portal.oit.ac.jp/CAMJWEB/slbssrch.do"},
    {"univ_name":"奈良大学","url":"https://portal.nara-u.ac.jp/portal/slbssrch.do"},
    {"univ_name":"同志社女子大学","url":"https://portal.nara-u.ac.jp/portal/slbssrch.do"},
    {"univ_name":"摂南大学","url":"https://portal.setsunan.ac.jp/CAMJWEB/slbsskgr.do?clearAccessData=true"},
    {"univ_name":"京都芸術大学","url":"https://n1.kyoto-art.ac.jp/campusweb/slbsskgr.do"},
    {"univ_name":"大阪体育大学","url":"https://syllabus.ouhs.ac.jp/campusweb/slbssrch.do"}
]
hard_coded_colleage_list =[
         {"univ_name":"京都芸術大学","url":"https://n1.kyoto-art.ac.jp/campusweb/slbsskgr.do"}]

In [3]:
#検索結果の表示ページを切り替える
def nexe_page(browser):

    sleep(5)
    next_button = browser.find_element(By.LINK_TEXT, '次の200件>>')   

    next_button.click()


def is_next_page_exist(browser):
    next_button = browser.find_element(By.LINK_TEXT, '次の200件>>')   
    if(next_button.get_attribute("disabled") == "true"):
        print("this is the last page.")
        return False
    else:
        return True

def record_links(browser,file_name):
    tbody = browser.find_element(By.TAG_NAME, "table").find_element(By.TAG_NAME, "tbody")
    tr_elements = tbody.find_elements(By.TAG_NAME, 'tr')
    for tr_element in tr_elements:
    # <input>要素を取得します
        try:
            even_td_elements = tr_element.find_elements(By.CLASS_NAME, 'column_even')
        except:
            #ヘッダー行の回避
            continue
        try:
            odd_td_elements = tr_element.find_elements(By.CLASS_NAME, 'column_odd')
        except:
            continue
        for td_element in even_td_elements:
            try:
                key = td_element.find_element(By.TAG_NAME, 'a').get_attribute("href")
            except:
                continue
        
            print(key)
            access_url = key
            with open(file_name, 'a') as f:
               f.write(access_url+"\n")

        for td_element in odd_td_elements:
            try:
                key = td_element.find_element(By.TAG_NAME, 'a').get_attribute("href")
            except:
                continue
        
            print(key)
            access_url = key
            with open(file_name, 'a') as f:
               f.write(access_url+"\n")


In [4]:
def extract_jp_from_string(text):
    match = re.search(r'\.jp', text)
    if match:
        jp_index = match.start() + 3  # 「.jp」の後ろの文字のインデックス
        jp_part = text[:jp_index]  # 「.jp」までの部分を抜き出す
        return jp_part
    else:
        return None


def append_line_to_file(filename, line):
    with open(filename, 'a') as file:
        file.write(line + '\n')

for column in hard_coded_colleage_list:
    new_dir_path = column['univ_name']
    new_record_path = './'+column['univ_name']+"/"+column['univ_name']+"_links.txt"
    search_url = column['url']
    base_url = extract_jp_from_string( column['url'])
    if not os.path.exists(new_dir_path):
        os.mkdir(new_dir_path)
    if not os.path.exists(new_record_path):
        f = open(new_record_path,'w')
        f.write("")
        f.close()
    browser1 = webdriver.Chrome()
    browser1.get(search_url)
    #search_button = browser1.find_element(By.NAME,'btnSearch')
    #search_button.click()
   

In [15]:

record_links(browser1,new_record_path)
while(is_next_page_exist(browser1)):
    nexe_page(browser1)
    record_links(browser1,new_record_path)
browser1.close()

https://n1.kyoto-art.ac.jp/campusweb/slbssbdr.do?value(risyunen)=2023&value(semekikn)=1&value(kougicd)=90001038&value(crclumcd)=200002
https://n1.kyoto-art.ac.jp/campusweb/slbssbdr.do?value(risyunen)=2023&value(semekikn)=1&value(kougicd)=90001045&value(crclumcd)=200002
https://n1.kyoto-art.ac.jp/campusweb/slbssbdr.do?value(risyunen)=2023&value(semekikn)=1&value(kougicd)=90001051&value(crclumcd)=200002
https://n1.kyoto-art.ac.jp/campusweb/slbssbdr.do?value(risyunen)=2023&value(semekikn)=1&value(kougicd)=90000006&value(crclumcd)=200004
https://n1.kyoto-art.ac.jp/campusweb/slbssbdr.do?value(risyunen)=2023&value(semekikn)=1&value(kougicd)=90000017&value(crclumcd)=200004
https://n1.kyoto-art.ac.jp/campusweb/slbssbdr.do?value(risyunen)=2023&value(semekikn)=1&value(kougicd)=90000031&value(crclumcd)=200004
https://n1.kyoto-art.ac.jp/campusweb/slbssbdr.do?value(risyunen)=2023&value(semekikn)=1&value(kougicd)=32201007&value(crclumcd)=201803
https://n1.kyoto-art.ac.jp/campusweb/slbssbdr.do?value(

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"次の200件>>"}
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x00000001062bf598 chromedriver + 4973976
1   chromedriver                        0x00000001062b6913 chromedriver + 4938003
2   chromedriver                        0x0000000105e73cd6 chromedriver + 470230
3   chromedriver                        0x0000000105eb72ff chromedriver + 746239
4   chromedriver                        0x0000000105eb7591 chromedriver + 746897
5   chromedriver                        0x0000000105ef9384 chromedriver + 1016708
6   chromedriver                        0x0000000105edb9ed chromedriver + 895469
7   chromedriver                        0x0000000105ef6856 chromedriver + 1005654
8   chromedriver                        0x0000000105edb793 chromedriver + 894867
9   chromedriver                        0x0000000105ea921f chromedriver + 688671
10  chromedri

In [1]:
import os
import glob
import random
import string
# カレントディレクトリのパスを取得
current_directory = os.getcwd()

# カレントディレクトリ内のフォルダを取得
folders = [f for f in os.listdir(current_directory) if os.path.isdir(os.path.join(current_directory, f))]

pattern = r')=(.*?)&value'
# フォルダの名前を表示
for folder_path in folders:
    file_paths = glob.glob(os.path.join(folder_path, "*.txt"))
    with open(file_paths[0], "r") as file:
        for line in file:
        # 行の処理を行うためのコードを追加
            response = requests.get(line)
            html = response.text
            match = re.search(pattern, line)
            if match:
                extracted_text = match.group(1)
                file_name = './' + folder_path + "/" + extracted_text + '.html'
            else:
                file_name = './'.join(random.choices(string.ascii_letters + string.digits, k=10)) + '.html'
            print(file_name)
            sleep(10)
            with open(r""+file_name, 'w', encoding='utf-8') as f:
                f.write(html)
                print(file_name)

NameError: name 'requests' is not defined